# FPsim Intervention Scenarios

With FPsim's user-friendly Scenarios API, we can easily setup and compare multiple intervention scenarios against one another to better understand the complex interactions between demographics and family planning behavior.

1. [Introduction](#introduction)
2. [Setup](#setup)
3. [Efficacy](#efficacy)
4. [Switching](#switching)
5. [Comparison](#comparison)

## Introduction <a name="introduction"></a>

FPsim is a useful tool for understanding how demographics, contraceptive efficacy, and contraceptive uptake behaviors interact with one another. We will be going through a couple of scenarios using our "Scenarios" api, which makes it easier for researchers to customize and compare the results of family planning interventions.

## Setup <a name="setup"></a>
#### A simple guide on how to set up a basic intervention scenario

The below steps will demonstrate how we can model an increase in modern contraceptive efficacy and visualize the results.

In [ ]:
import sciris as sc
import fpsim as fp

In [ ]:
# First we create a Scenario object where:
#    eff is the dictionary of specifications for how we want to change efficacy
#    year is the year we want to implement the change
#    label is how we want to label the scenario in the visualization
scenario = fp.make_scen(eff={'Other modern':0.994}, year=2005, label='Increased modern')

# Then we add it to the Scenarios object which is used to compare results across multiple scenarios
# but for this case we will just be visualizing the one. Repeats defines how many times to run the 
# simulation in order to determine uncertainty intervals.
scenarios = fp.Scenarios(scens=scenario, repeats=3)

# Now that we have the scenario specifications, we can run the simulations
scenarios.run()

# To examine the resulting demoraphic features of the scenario we can use the plot_scens() function
scenarios.plot_scens()

## Efficacy <a name="efficacy"></a>
#### How to to compare sim results across different contraceptive efficacies

Let's suppose we want to better understand how demographic features change at varying levels of efficacy of modern contraceptive methods. For this task we would create three specification dictionaries, add them to scenarios, and then run.

In [ ]:
# In order to compare different scenarios to one another we must:

# Define the different scenarios
modern_995 = fp.make_scen(eff={'Other modern':0.994}, year=2005, label='Modern high efficacy')

modern_75 = fp.make_scen(eff={'Other modern':0.75}, year=2005, label='Modern moderate efficacy')

modern_50 = fp.make_scen(eff={'Other modern':0.5}, year=2005, label='Modern low efficacy')

# Add to scenarios object
scens_eff = fp.Scenarios(n=1000, repeats=3, scens=[modern_995, modern_75, modern_50])

# Run scenarios
scens_eff.run()

# Compare results
scens_eff.plot_scens()

## Switching <a name="switching"></a>
#### How to compare sim results across different switching probabilities.

Let's suppose a researcher wants to understand how an increase in a preference for condoms over other modern methods would affect cumulative live births. Below is how we would do this using Scenarios:

In [ ]:
# Within our scenario we can adjust the probability of switching from one intervention (source) to another (dest).
# For the purpose of granular analysis we can even target specific ages, and postpartum status which often affects
# a women's contraceptive uptake behavior. In the below example we target women in the first six months postpartum (matrix).
high_switching = fp.make_scen(source='Other modern', dest='Injectables', year=2005, value=0.95, ages='18-25', matrix='pp1to6', label="high switching")
medium_switching = fp.make_scen(source='Other modern', dest='Injectables', year=2005, value=0.75, ages='18-25', matrix='pp1to6', label="medium switching")
low_switching = fp.make_scen(source='Other modern', dest='Injectables', year=2005, value=0.50, ages='18-25', matrix='pp1to6', label="low switching")

# Then we pass these scenarios into the Scenarios object as before
scens_switching = fp.Scenarios(n=100, repeats=3, scens=[high_switching, medium_switching, low_switching])
scens_switching.run()

scens_switching.plot_scens()

## Comparison <a name="comparison"></a>
#### How to compare sim results in different ways

The Scenarios object has a number of methods for plotting results across different scenarios. Besides plot_scens() we also have **plot_sims()** which shows us the results of each individual sim run, **plot_cpr()** which shows us some statistics on CPR, and **analyze_sims()** which logs the results of the run as a handy pandas DataFrame.

**plot_sims()**

Plot sims is useful for when we want to understand the specific results of each individual sim for a given scenario. Although a group of sims for a single scenario should all have the same parameters, a difference in seed can produce a significant effect.

In [ ]:
# Single scenario with 3 runs
scenarios.plot_sims()

**plot_cpr()**
If we are more interested in the relationship between contraceptive usage and age, then we can utilize this function to narrow our focus. Namely it offers three visualizations to understand the prevalence of contraceptive use:
1. **mCPR** - modern contraceptive users who are women between 15-49
2. **CPR** - all contraceptive users who are women between 15-49
3. **ACPR** - all contraceptive users who are women that are sexually active and not pregnant


In [ ]:
scenarios.plot_cpr()

**analyze_sims()**

Lastly, if we want to create our own custom visualization, analyze_sims() let's us log the data as a dictionary or dataframe.

In [ ]:
scenarios.analyze_sims()

In [ ]:
# Dataframe format let's use the built-in dataframe functions for an aggregate view of the results
df = scenarios.results.df
df.plot(kind='bar', x="scenario", y="popsize")

In [ ]:
# If we want some summary statistics on the results of the scenarios, we can view the results as a dictinary using .stats
scens_eff.results.stats

## More info
### For more info check out the [source code](https://github.com/amath-idm/fp_analyses) 